<a href="https://www.kaggle.com/code/juancmoreno1508/red-neuronal?scriptVersionId=256699372" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

print(torch.cuda.is_available())  # Debe dar True
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


# Transformaciones para normalizar y convertir las imágenes a tensores
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Descarga y carga del conjunto de datos de entrenamiento
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=4, pin_memory=True)

# Descarga y carga del conjunto de datos de prueba
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2, pin_memory=True)

# Clases del dataset
classes = ('Avion', 'carro', 'Pajaro', 'Gato', 'Ciervo', 'Perro', 'Sapo', 'Caballo', 'Barco', 'Camion')

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # Desnormalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Obtener un lote de imágenes de entrenamiento
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Mostrar las imágenes
imshow(torchvision.utils.make_grid(images))
# Imprimir las etiquetas de las imágenes
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # aplanar todas las dimensiones excepto el batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)
print(net)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
import time

for epoch in range(20):
    start_time = time.time()  # Iniciar el cronómetro para la época
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Obtener los inputs; `data` es una lista de [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # Poner los gradientes a cero
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Imprimir estadísticas
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    
    end_time = time.time() # Detener el cronómetro
    elapsed_time = end_time - start_time
    print(f'Epoch {epoch + 1} finished in {elapsed_time:.2f} seconds') # Imprimir el tiempo de la época

print('Finished Training')

In [ ]:
correct = 0
total = 0
# No necesitamos calcular gradientes en la fase de prueba
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # Calcular los resultados del modelo
        outputs = net(images)
        # La clase con la energía más alta es lo que predecimos
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Precision de la red en 10000 test de imagen: {100 * correct // total} %')

In [ ]:
# Guardar el modelo en el disco
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)